# <CENTER>  Read Data

In [1]:
import os
import json
import gzip
import pandas as pd
import numpy as np

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

# <CENTER> Read file

In [3]:
df_M_TG = getDF(r"E:\NIIT\Project\DATA_2014\T&G\meta_Toys_and_Games.json.gz")

In [4]:
df_R_TG = getDF(r"E:\NIIT\Project\DATA_2014\T&G\reviews_Toys_and_Games.json.gz")

In [5]:
df_5_TG = getDF(r"E:\NIIT\Project\DATA_2014\T&G\reviews_Toys_and_Games_5.json.gz")

# <CENTER> Data preprocessing

In [16]:
df_M_TG.isnull().sum()

asin                0
description     33285
title             898
price           93381
salesRank        8849
imUrl             350
brand          150706
categories          0
related         71687
dtype: int64

## Exporting various variables to understand

In [40]:
file_name = "description_m_TG14.txt"
descriptions = df_M_TG['description']
with open(file_name, 'w', encoding='utf-8') as f:
    for description in descriptions:
        f.write(str(description) + '\n')

In [43]:
file_name = "salesRank_m_TG14.txt"
salesRank = df_M_TG['salesRank']
with open(file_name, 'w', encoding='utf-8') as f:
    for salesRank in salesRank:
        f.write(str(salesRank) + '\n')

In [7]:
file_name = "title.txt"
title = df_M_TG['title']
with open(file_name, 'w', encoding='utf-8') as f:
    for title in title:
        f.write(str(title) + '\n')

## Creating copy of data

In [8]:
df_temp_MTG=df_M_TG.copy()

In [9]:
df_temp_MTG.shape

(336072, 9)

## Renaming the 'asin' and deoping the unnecessary data 

In [17]:
df_temp_MTG = df_temp_MTG.rename(columns={'asin': 'ProductID'})

In [18]:
len(df_temp_MTG['ProductID'].value_counts())

336072

In [19]:
df_temp_MTG.shape

(336072, 9)

In [20]:
df_temp_MTG=df_temp_MTG.drop(['imUrl','brand','salesRank','description'],axis=1)

## Remove Characters "{},(),[]"

In [21]:
import re 
def removechar(text):
    special_chars_pattern = r'[\{\}\(\)\[\]]'
    cleaned_text = re.sub(special_chars_pattern, '', text)
    return cleaned_text

In [22]:
C_list=['categories','related']
df_temp_MTG[C_list] = df_temp_MTG[C_list].astype(str)
for col in C_list:
    df_temp_MTG[col] = df_temp_MTG[col].apply(removechar)

In [23]:
df_temp_MTG.head()

,ProductID,title,price,categories,related
0,0000191639,Dr. Suess 19163 Dr. Seuss Puzzle 3 Pack Bundle,37.12,"'Toys & Games', 'Puzzles', 'Jigsaw Puzzles'",nan
1,0005069491,Nursery Rhymes Felt Book,NaN,'Toys & Games',nan
2,0076561046,Fraction Decimal Percent Card Deck,NaN,"'Toys & Games', 'Learning & Education', 'Flash...",'also_viewed': '0075728680'
3,0131358936,NaN,36.22,"'Toys & Games', 'Learning & Education', 'Mathe...","'also_bought': '0321845536', '0078787572', 'bu..."
4,0133642984,Algebra 2 California Teacher Center,731.93,"'Toys & Games', 'Learning & Education', 'Mathe...",nan


In [ ]:
file_name = "categories_mtg14.txt"
categories = df_temp_MTG['categories']
with open(file_name, 'w', encoding='utf-8') as f:
    for categories in categories:
        f.write(str(categories) + '\n')

## Creating sub-Category from category

In [24]:
Temp_var_mtg=df_temp_MTG["categories"].str.split(",", n = 1, expand = True)

In [25]:
df_temp_MTG['Category']=Temp_var_mtg[0]

In [26]:
df_temp_MTG['temp_cat']=Temp_var_mtg[1]

In [27]:
Temp_var_mtg2=df_temp_MTG["temp_cat"].str.split(",", n = 1, expand = True)

In [28]:
df_temp_MTG['Sub-category']=Temp_var_mtg2[0]
df_temp_MTG=df_temp_MTG.drop(['temp_cat','categories'],axis=1)

In [29]:
df_temp_MTG.head(5)

,ProductID,title,price,related,Category,Sub-category
0,0000191639,Dr. Suess 19163 Dr. Seuss Puzzle 3 Pack Bundle,37.12,nan,'Toys & Games','Puzzles'
1,0005069491,Nursery Rhymes Felt Book,NaN,nan,'Toys & Games',None
2,0076561046,Fraction Decimal Percent Card Deck,NaN,'also_viewed': '0075728680','Toys & Games','Learning & Education'
3,0131358936,NaN,36.22,"'also_bought': '0321845536', '0078787572', 'bu...",'Toys & Games','Learning & Education'
4,0133642984,Algebra 2 California Teacher Center,731.93,nan,'Toys & Games','Learning & Education'


In [30]:
df_temp_MTG['Sub-category'].fillna("'Toys & Games'",inplace=True)

In [31]:
df_temp_MTG.isnull().sum()

ProductID           0
title             898
price           93381
related             0
Category            0
Sub-category        0
dtype: int64

In [32]:
df_temp_MTG['title'].fillna("Toys&Games",inplace=True)

In [33]:
Temp_var_mtg3=df_temp_MTG["related"].str.split(":", n = 1, expand = True)

In [34]:
df_temp_MTG['also_buy']=Temp_var_mtg3[0]
df_temp_MTG['other']=Temp_var_mtg3[1]

## Converting "also_buy" column to binary as "yes/no"

In [35]:
for i in range(len(df_temp_MTG['also_buy'])):
    if df_temp_MTG['also_buy'][i] == "'also_bought'":
        df_temp_MTG['also_buy'][i] = '1'
    else:
        df_temp_MTG['also_buy'][i] = '0'

C:\Users\Rishabh\AppData\Local\Temp\ipykernel_14052\4159863847.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp_MTG['also_buy'][i] = '0'
C:\Users\Rishabh\AppData\Local\Temp\ipykernel_14052\4159863847.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp_MTG['also_buy'][i] = '1'


In [36]:
df_temp_MTG=df_temp_MTG.drop(['related','other'],axis=1)

In [37]:
def remove_quotes(text):
    return text.replace("'", '')

df_temp_MTG['Sub-category'] = df_temp_MTG['Sub-category'].apply(remove_quotes)
df_temp_MTG['Category'] = df_temp_MTG['Category'].apply(remove_quotes)

In [38]:
df_temp_MTG['also_buy']=df_temp_MTG['also_buy'].astype(int)

In [113]:
df_temp_MTG.dtypes

ProductID        object
title            object
price           float64
Category         object
Sub-category     object
also_buy          int32
dtype: object

In [40]:
df_sort_MTG=df_temp_MTG.copy()

In [102]:
df_sort_MTG.head()

,ProductID,title,price,Category,Sub-category,also_buy
0,0000191639,Dr. Suess 19163 Dr. Seuss Puzzle 3 Pack Bundle,37.12,Toys & Games,Puzzles,0
1,0005069491,Nursery Rhymes Felt Book,15.66,Toys & Games,Toys & Games,0
2,0076561046,Fraction Decimal Percent Card Deck,3.99,Toys & Games,Learning & Education,0
3,0131358936,Toys&Games,36.22,Toys & Games,Learning & Education,1
4,0133642984,Algebra 2 California Teacher Center,731.93,Toys & Games,Learning & Education,0


In [104]:
len(df_sort_MTG['Sub-category'].unique())

72

In [43]:
from sklearn.preprocessing import LabelEncoder

# Assuming df_sort_MVG is your DataFrame
sub_categories = df_sort_MTG['Sub-category'].unique()

# Initialize the LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(sub_categories)

LabelEncoder()

In [44]:
df_sort_MTG['Sub-category_encoded'] = label_encoder.transform(df_sort_MTG['Sub-category'])

In [45]:
df_sort_MTG.sample(10)

,ProductID,title,price,Category,Sub-category,also_buy,Sub-category_encoded
330006,B00HSVPSQ0,Dragon Ball Z Ss3 Goku High Grade Revival PVC ...,NaN,Toys & Games,Action Figures & Statues,0,3
85202,B0014FIO12,Ugly Doll Babo's Bird 4&quot; Ugly Doll Clip-On,3.20,Toys & Games,Stuffed Animals & Plush,1,65
282096,B00A9NKHY2,3D Semi-circular Black and White Checkered Sku...,2.99,Toys & Games,Arts & Crafts,1,5
158106,B003EH3GV6,Mighty Wheels Army Humvee Truck Diecast,NaN,Toys & Games,Toy Remote Control & Play Vehicles,1,69
311620,B00DYFB3HW,Disney Doc McStuffins Sticker Sheets 4 Pack,2.00,Toys & Games,Arts & Crafts,1,5
145826,B0030PIVDU,ZhuZhu GIANT Hamster FUN House (hamsters sold ...,NaN,Toys & Games,Action Figures & Statues,0,3
63732,B000Q1FD4Q,Yoda Dashboard Driver,NaN,Toys & Games,Toy Remote Control & Play Vehicles,0,69
141626,B002VQ457A,Pack of 12 Zoo Animal Shaped Rubber Bands,2.49,Toys & Games,Dress Up & Pretend Play,0,22
45213,B000H3N8OA,Truck Driver,NaN,Toys & Games,Dolls & Accessories,0,21
240586,B0072H7MN2,100% Catholic Iron On Embroidered Applique Pat...,NaN,Toys & Games,Arts & Crafts,1,5


In [46]:
df_sort_MTG.set_index('Sub-category_encoded', inplace=True)

## Filling price variable values using interpolate function

In [47]:
df_sort_MTG['price'] = df_sort_MTG['price'].interpolate(method='index', limit_direction="both")

In [48]:
df_sort_MTG.reset_index(inplace=True)
df_sort_MTG.drop('Sub-category_encoded',axis=1,inplace=True)

In [115]:
len(df_sort_MTG['ProductID'].value_counts())

336072

In [117]:
replaced_values = df_sort_MTG[df_sort_MTG['price'] != df_temp_MTG['price']]
len(replaced_values['price'].value_counts())

40

In [50]:
df_sort_MTG.to_csv('T&G_EDA_Data.csv', index=False)

# --------------------

# <CENTER>  5-core Data

In [118]:
df_5_TG.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1VXOAVRGKGEAK,0439893577,Angie,"[0, 0]",I like the item pricing. My granddaughter want...,5.0,Magnetic board,1390953600,"01 29, 2014"
1,A8R62G708TSCM,0439893577,Candace,"[1, 1]",Love the magnet easel... great for moving to d...,4.0,it works pretty good for moving to different a...,1395964800,"03 28, 2014"


In [119]:
df_5_TG.shape

(167597, 9)

In [32]:
# given_reviewer_id = ["A1VXOAVRGKGEAK"]
# is_present = given_reviewer_id in df_R_TG['reviewerID'].values

# if is_present:
#     print(f"The reviewer ID {given_reviewer_id} is present in df_R_TG.")
# else:
#     print(f"The reviewer ID {given_reviewer_id} is not present in df_R_TG.")

The reviewer ID ['A1VXOAVRGKGEAK'] is present in df_R_TG.


In [37]:
# P_count = 0
# NP_count = 0
# for i in df_5_TG['reviewerID'].values[:100]:
#     if i in df_R_TG['reviewerID'].values:
#         P_count += 1
#     else:
#         NP_count += 1
# print('Total present:',P_count)
# print('Total Not present:',NP_count)

Total present: 100
Total Not present: 0


In [120]:
len(df_5_TG['asin'].value_counts())

11924

In [124]:
df_5_TG['reviewerID'].value_counts()

AJGU56YG8G1DQ     550
A1M8AYAL3L8ACP    168
A1II2ZRPKZAQQD    157
A2WW57XX2UVLM6    151
A1POFVVXUZR3IQ    133
                 ... 
A3NLQWUUML5S7F      5
A2AWAXPNKJT5TE      5
A1GQ1JGK9YM6BU      5
A2Q7M6BFUX21L9      5
A1KQNP8MOJDJKC      5
Name: reviewerID, Length: 19412, dtype: int64

In [121]:
df_temp_5TG=df_5_TG.copy()

In [126]:
df_temp_5TG.head(2)

,reviewerID,ProductID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1VXOAVRGKGEAK,0439893577,Angie,"[0, 0]",I like the item pricing. My granddaughter want...,5.0,Magnetic board,1390953600,"01 29, 2014"
1,A8R62G708TSCM,0439893577,Candace,"[1, 1]",Love the magnet easel... great for moving to d...,4.0,it works pretty good for moving to different a...,1395964800,"03 28, 2014"


In [138]:
grouped_df = df_R_TG.groupby('reviewerID').agg({
    'asin': 'count',
    'reviewerName':'count',
    'helpful':'count',
    'reviewText':'count',
    'overall':'count',
    'summary':'count',
    'unixReviewTime':'count',
    'reviewTime':'count'
}).reset_index()

grouped_df.sort_values('asin',ascending=False)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
1180036,AJGU56YG8G1DQ,1528,1527,1528,1528,1528,1528,1528,1528
1254040,AQZH7YTWQPOBE,570,570,570,570,570,570,570,570
679830,A2WW57XX2UVLM6,488,488,488,488,488,488,488,488
62374,A168O2YKPE9BE8,409,409,409,409,409,409,409,409
272789,A1RKCT4H3X3J1W,407,407,407,407,407,407,407,407
...,...,...,...,...,...,...,...,...,...
517286,A2GD8F3D9741Q2,1,1,1,1,1,1,1,1
517287,A2GD8JVPU0YYSK,1,1,1,1,1,1,1,1
517288,A2GD8P9MG59G0A,1,1,1,1,1,1,1,1
517289,A2GD8PQ4RCC0HA,1,1,1,1,1,1,1,1


In [135]:
grouped_df = df_temp_5TG.groupby('reviewerID').agg({
    'ProductID': 'count',
    'reviewerName':'count',
    'helpful':'count',
    'reviewText':'count',
    'overall':'count',
    'summary':'count',
    'unixReviewTime':'count',
    'reviewTime':'count'
}).reset_index()

grouped_df.sort_values('ProductID',ascending=False)

,reviewerID,ProductID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
17012,AJGU56YG8G1DQ,550,549,550,550,550,550,550,550
3140,A1M8AYAL3L8ACP,168,167,168,168,168,168,168,168
2587,A1II2ZRPKZAQQD,157,155,157,157,157,157,157,157
9812,A2WW57XX2UVLM6,151,151,151,151,151,151,151,151
3617,A1POFVVXUZR3IQ,133,133,133,133,133,133,133,133
...,...,...,...,...,...,...,...,...,...
6677,A2B3GYUQEAI8XK,5,5,5,5,5,5,5,5
6679,A2B44JHPZ1U92G,5,5,5,5,5,5,5,5
13990,A3QBB4VPEBJJ24,5,5,5,5,5,5,5,5
13988,A3QAQCG2OBJ9NA,5,5,5,5,5,5,5,5


## Changing 'asin' to 'ProductID'

In [122]:
df_temp_5TG = df_temp_5TG.rename(columns={'asin': 'ProductID'})

In [139]:
df_temp_5TG.isnull().sum()

reviewerID          0
ProductID           0
reviewerName      838
helpful             0
reviewText          0
overall             0
summary             0
unixReviewTime      0
reviewTime          0
dtype: int64

In [141]:
df_temp_5TG['reviewerName'].fillna('Not_available',inplace=True)

In [142]:
df_temp_5TG.head()

,reviewerID,ProductID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1VXOAVRGKGEAK,0439893577,Angie,"[0, 0]",I like the item pricing. My granddaughter want...,5.0,Magnetic board,1390953600,2014-01-29
1,A8R62G708TSCM,0439893577,Candace,"[1, 1]",Love the magnet easel... great for moving to d...,4.0,it works pretty good for moving to different a...,1395964800,2014-03-28
2,A21KH420DK0ICA,0439893577,capemaychristy,"[1, 1]",Both sides are magnetic. A real plus when you...,5.0,love this!,1359331200,2013-01-28
3,AR29QK6HPFYZ4,0439893577,dcrm,"[0, 0]",Bought one a few years ago for my daughter and...,5.0,Daughters love it,1391817600,2014-02-08
4,ACCH8EOML6FN5,0439893577,DoyZ,"[1, 1]",I have a stainless steel refrigerator therefor...,4.0,Great to have so he can play with his alphabet...,1399248000,2014-05-05


## Converting reviewtime to Datetime 

In [140]:
from datetime import datetime

date_format = "%m %d, %Y"
df_temp_5TG['reviewTime'] = pd.to_datetime(df_temp_5TG['reviewTime'], format=date_format).dt.date

In [143]:
df_temp_5TG = df_temp_5TG.rename(columns={'reviewTime': 'Date'})

In [144]:
df_temp_5TG.head(2)

,reviewerID,ProductID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,Date
0,A1VXOAVRGKGEAK,0439893577,Angie,"[0, 0]",I like the item pricing. My granddaughter want...,5.0,Magnetic board,1390953600,2014-01-29
1,A8R62G708TSCM,0439893577,Candace,"[1, 1]",Love the magnet easel... great for moving to d...,4.0,it works pretty good for moving to different a...,1395964800,2014-03-28


In [145]:
df_temp_5TG.drop('unixReviewTime',axis=1,inplace=True)

In [146]:
df_temp_5TG['text']=df_temp_5TG['reviewText']+df_temp_5TG['summary']

In [147]:
df_temp_5TG.drop(['reviewText','summary','helpful'],axis=1,inplace =True)

In [148]:
df_temp_5TG.head()

,reviewerID,ProductID,reviewerName,overall,Date,text
0,A1VXOAVRGKGEAK,0439893577,Angie,5.0,2014-01-29,I like the item pricing. My granddaughter want...
1,A8R62G708TSCM,0439893577,Candace,4.0,2014-03-28,Love the magnet easel... great for moving to d...
2,A21KH420DK0ICA,0439893577,capemaychristy,5.0,2013-01-28,Both sides are magnetic. A real plus when you...
3,AR29QK6HPFYZ4,0439893577,dcrm,5.0,2014-02-08,Bought one a few years ago for my daughter and...
4,ACCH8EOML6FN5,0439893577,DoyZ,4.0,2014-05-05,I have a stainless steel refrigerator therefor...


## Text preprocessing using SPACY

In [149]:
### utlity function for pre-processing the text
import spacy
import re

def remove_special_characters(text):
    # Remove all non-letters and non-spaces except for hyphens and digits
    text = re.sub("[^0-9A-Za-z\- ]+", " ", text)
    # Remove unnecessary symbols, punctuation, and signs
    text = re.sub(r'[^\w\s]', '', text)
    # Remove all numbers except those attached to a word
    text = re.sub("(?<!\w)\d+", "", text)
    # Remove all hyphens except between two words
    text = re.sub("-(?!\w)|(?<!\w)-", "", text)
    # Remove multiple spaces
    text = " ".join(text.split())
    return text

In [150]:
nlp = spacy.load("en_core_web_sm")
# code with spacy
# def preprocess(text):
#     if text is not np.nan:
#         # Lowercase everything
#         text = text.lower()
#         # Lemmatize the text
#         doc = nlp(text)
#         filtered_tokens = []
#         for token in doc:
#             if token.is_stop:
#                 continue
#             filtered_tokens.append(token.lemma_)
#         # Remove extra spaces
#         return " ".join(filtered_tokens).strip()

## Text preprocessing using NLTK

In [151]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Load the NLTK lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    if text is not np.nan:
        # Lowercase everything
        text = text.lower()
        # Tokenize the text
        tokens = nltk.word_tokenize(text)
        filtered_tokens = []
        for token in tokens:
            if token in stop_words:
                continue
            # Lemmatize the token
            lemmatized_token = lemmatizer.lemmatize(token)
            filtered_tokens.append(lemmatized_token)
        # Remove extra spaces
        return " ".join(filtered_tokens).strip()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rishabh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rishabh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rishabh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [152]:
df_temp_5TG['text']=df_temp_5TG['text'].apply(remove_special_characters)

In [153]:
df_temp_5TG['text']=df_temp_5TG['text'].apply(preprocess)

In [154]:
df_temp_5TG.head()

,reviewerID,ProductID,reviewerName,overall,Date,text
0,A1VXOAVRGKGEAK,0439893577,Angie,5.0,2014-01-29,like item pricing granddaughter wanted mark wa...
1,A8R62G708TSCM,0439893577,Candace,4.0,2014-03-28,love magnet easel great moving different area ...
2,A21KH420DK0ICA,0439893577,capemaychristy,5.0,2013-01-28,side magnetic real plus entertaining one child...
3,AR29QK6HPFYZ4,0439893577,dcrm,5.0,2014-02-08,bought one year ago daughter love still using ...
4,ACCH8EOML6FN5,0439893577,DoyZ,4.0,2014-05-05,stainless steel refrigerator therefore much sp...


In [155]:
df_temp_5TG.to_csv('T&G_5TG_review.csv', index=False)

In [137]:
df_sort_MTG.head()

,ProductID,title,price,Category,Sub-category,also_buy
0,0000191639,Dr. Suess 19163 Dr. Seuss Puzzle 3 Pack Bundle,37.12,Toys & Games,Puzzles,0
1,0005069491,Nursery Rhymes Felt Book,15.66,Toys & Games,Toys & Games,0
2,0076561046,Fraction Decimal Percent Card Deck,3.99,Toys & Games,Learning & Education,0
3,0131358936,Toys&Games,36.22,Toys & Games,Learning & Education,1
4,0133642984,Algebra 2 California Teacher Center,731.93,Toys & Games,Learning & Education,0


In [143]:
len(df_temp_5TG['ProductID'].value_counts())

11924

In [145]:
len(df_sort_MTG['ProductID'].value_counts())

336072

## Merging both data 

In [152]:
TG_df = df_sort_MTG.merge(df_temp_5TG, on='ProductID', how='inner')

In [153]:
TG_df.shape

(167597, 11)

In [154]:
TG_df.isnull().sum()

ProductID       0
title           0
price           0
Category        0
Sub-category    0
also_buy        0
reviewerID      0
reviewerName    0
overall         0
Date            0
text            0
dtype: int64

In [155]:
len(TG_df['ProductID'].value_counts())

11924

# Uploading File as CSV format

In [62]:
TG_df.to_csv('T&G_Data.csv', index=False)

# ---------------------------------------